In [ ]:
import sys
sys.path.insert(0, '../tools')

from pprint import pprint
%load_ext prov_magics

import prov_dictionary
from prov_parser import build_parser
from qversioned_prov import *

In [ ]:
from dot import _unquote

def extract_entity(eid, attrs={}, id_=None):
    return ["entity", [eid], attrs, id_]

def extract_value(vid, attrs={}, id_=None):
    return ["value", [vid], attrs, id_]


parser = build_parser({
    'prefix': lambda *args, **kwargs: None,
    'entity': extract_entity,
    'value': extract_value,
})



def ext(text, arg):
    node = parser(text)[0]
    if arg.startswith("_"):
        return node[1][int(arg[1:])]
    if arg == "id_":
        return node[-1]
    if arg in node[2]:
        return node[2][arg]
    return None

def attr(text, arg):
    return _unquote(ext(text, arg))
            
def update(text, **kwargs):
    node = parser(text)[0]
    for arg, value in kwargs.items():
        if arg.startswith("_"):
            node[1][int(arg[1:])] = value
        elif arg == "id_":
            node[-1] = value
        elif arg.startswith("d__"):
            del node[2][arg]
        else:
            node[2][arg] = value
    return querier.text(*node)



In [ ]:
querier.load("../mutable_prov/temp/L2__list_definition0.provn");
querier.generate('''
value(v10000, [repr="10000"])
entity(e10000, [value="10000", type="literal"])
defined(e10000, v10000, 2018-02-26T14:41:52.080421)
''')

In [21]:

from copy import copy
from collections import defaultdict
from collections import deque
import bisect
import datetime

class StateController(dict):
    """Helper dictionary with functions to get the biggest time smaller or equals to a time"""
    def __init__(self):
        self.order = []
        
    def __setitem__(self, time, obj):
        if time not in self:
            bisect.insort(self.order, time)

        super(StateController, self).__setitem__(time, obj)
        
    def find_time(self, time):
        index = bisect.bisect_right(self.order, time) - 1
        if index == -1:
            index = 0
        return self.order[index]
    
    def find(self, time):
        return self[self.find_time(time)]


def build_original_reference_map():
    """Follow wasDerivedFrom(C, E, -, -, -, [type="Reference"]) until entity that has the membership"""
    C, E = var("C E")
    same = {}
    for __ in was_derived_from(C, E, type="Reference"):
        same[C.bound] = E.bound

    new_same = copy(same)
    for key in same:
        current = new_same[key]
        next_key = new_same.get(current)
        while next_key:
            current = next_key
            next_key = new_same.get(current)
        new_same[key] = current
    return new_same

def rebuild_all_states(same):
    """Follow hadMember relationships to rebuild states
    
    Navigate on the hadMember graph to rebuild states of sub-entities
    """
    C, E, K, T, TX = var("C E K T TX")
    # Create all_states simple graph
    all_states = defaultdict(StateController)
    parents = defaultdict(set)
    derivations = defaultdict(list)
    for __ in had_member(C, E, key=K, moment=T):
        derivations[C.bound].append([T.bound, E.bound, K.bound])
    for val, der in derivations.items():
        der.sort()
        obj = {}
        for time, entity, key in der:
            obj = copy(obj)
            obj[key] = same.get(entity, entity)
            all_states[val][time] = obj
        
    # Navigate on the graph
    queue = deque(all_states.keys())
    visited = set()
    while queue:
        current = queue.pop()
        if current in visited:
            continue
        processed = True
        keys = all_states[current].order + [datetime.datetime.max]
        it = iter(keys)
        next(it)
        for time1, time2 in zip(keys, it):
            obj = all_states[current][time1]
            for value in obj.values():
                if value in all_states:
                    if value in visited:
                        # Get changes that occurred between two states
                        for time in all_states[value]:
                            if time1 <= time <= time2:
                                all_states[current][time] = obj
                                parents[(same.get(value, value), time)].add(current)
                    else:
                        processed = False
        if processed:
            visited.add(current)
        else:
            queue.appendleft(current)
    return all_states, parents

    


In [16]:
querier.load("../images/versioned_prov/floydwarshall.provn");

In [22]:
same = build_original_reference_map()
all_states, parents = rebuild_all_states(same)
all_states['list'].find(datetime.datetime.max)
parents

defaultdict(set,
            {('list0',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): {'list'},
             ('list0',
              datetime.datetime(2018, 3, 5, 16, 4, 23, 958634)): {'list'},
             ('list1',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): {'list'},
             ('list1',
              datetime.datetime(2018, 3, 5, 16, 4, 28, 133446)): {'list'},
             ('list2',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): {'list'},
             ('list2',
              datetime.datetime(2018, 3, 5, 16, 4, 22, 139873)): {'list'}})

In [32]:
visited = set()
todo = defaultdict(list)
_ = BLANK
A, D, E, T, W, U, TX, TX2 = var("A D E T W U TX TX2")
for __ in was_derived_from(D, E, A, _, _, moment=T, collection=W, access="w", text=TX):
    wsame = same.get(W.bound, W.bound)
    # use previous time to get used collection
    time = all_states[wsame].find_time(T.bound - datetime.timedelta(microseconds=1))
    state = (W.bound, time)  
    for __ in used(_, A.bound, W.bound, text=TX2):
        todo[state].append(("used", TX2.bound))
        visited.add(TX2.bound)
    
todo

defaultdict(list,
            {('disti#3',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): [('used',
               'used(assign18, disti#3, -)')],
             ('disti#4',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): [('used',
               'used(assign25, disti#4, -)')],
             ('disti#8',
              datetime.datetime(2018, 3, 5, 16, 4, 18, 294847)): [('used',
               'used(assign41, disti#8, -)')]})

In [ ]:
a = 4

In [ ]:
eval("a", {"a": 4})

In [ ]:
result = set()
todo = defaultdict(list)
shared = defaultdict(list)

def find_non_none(exp, *args):
    

def find_closest_future(enti, activity, time=None, last=datetime.datetime.max):
    if time:
        return time
    _ = BLANK
    A, E, T, T2, W = var("A E T T2 W")
    entities = []
    for __ in used(_, activity, _, T):
        if T.bound:
            return T.bound
    for __ in was_generated_by(_, E, activity, T) & entity(E, T2):
        if T.bound:
            return T.bound
        if T2.bound:
            return T2.bound - datetime.timedelta(microseconds=1)
        entities.append(E.bound)
    for __ in was_derived_from(_, _, activity, _, _, collection=W, moment=T):
        if T.bound:
            if W.bound == enti:
                return T.bound - datetime.timedelta(microseconds=1)
            return T.bound
    for ent in entities:
        for __ in used(_, A, ent, T):
            if T.bound:
                return T.bound
            result = find_closest_future(enti, A.bound, last=None)
            if result:
                return result
        for __ in was_derived_from(_, ent, A, _, _, moment=T):
            if T.bound:
                return T.bound
            result = find_closest_future(enti, A.bound, last=None)
            if result:
                return result
    return last
    
    

_ = BLANK
A, E, T, TX = var("A E T TX")
for __ in used(_, A, E, T, text=TX):
    esame = same.get(E.bound, E.bound) 
    if esame not in all_states:
        result.add(TX.bound)
    else:
        time = find_closest_future(E.bound, A.bound, T.bound)
        state = (E.bound, all_states[esame].find_time(time))
        todo[state].append(("used", TX.bound))
        estate = (esame, state[1])
        shared[estate].append(state)

shared
    

In [ ]:
import importlib
import annotations as prov
importlib.reload(prov)

def add_unmarked(marked, fn):
    TX = var()
    for _ in fn(text=TX):
        text = TX.bound
        if text not in marked:
            prov.add(text)

prov.reset_prov("../mutable_prov/unfold/", False)


marked = set()
E, V, C, T, TX = var(5)
valuemap = {
    V.bound: TX.bound
    for _ in value(V, TX)
}

value_with_parts = {
    V.bound: (E.bound, V.bound, C.bound, T.bound, TX.bound)
    for _ in derived_by_insertion(E, V, C, T, TX)
}

value_without_parts = set(valuemap) - set(value_with_parts) 

value_to_entity = {}

# Add imutable values
for vid in value_without_parts:
    vtext = valuemap[vid]
    for _ in entity(E, TX) & accessed(E, vid):
        marked.add(TX.bound)
        prov.entity(ext(TX.bound, "_0"), attr(vtext, "repr"), attr(TX.bound, "type"))
        value_to_entity[vid] = E.bound
    for _ in entity(E, TX) & defined(E, vid):
        marked.add(TX.bound)
        prov.entity(ext(TX.bound, "_0"), attr(vtext, "repr"), attr(TX.bound, "type"))
        value_to_entity[vid] = E.bound
    
# List definition
for _ in defined(E, V, T) & derived_by_insertion(E, V, C, T) & entity(E, TX):
    vtext = valuemap[V.bound]
    list_id = ext(TX.bound, "_0")
    list_value = attr(vtext, "repr")
    der = []
    parts = {}
    collection = prov.entity(list_id, list_value, "Dictionary")
    empty = prov.entity("e" + list_id, "{}", "EmptyDictionary")
    for key, vid in C.bound:
        key = _unquote(key)
        member = prov.entity(
            list_id[:-1] + key + "_" + list_id[-1], 
            attr(valuemap[vid], "repr"), "number")
        der.append((member, value_to_entity[vid]))
        parts[key] = member
    
    prov.derivedByInsertionFrom(collection, empty, parts)
    prov.activity("definelist", der, generated=[collection])
    marked.add(TX.bound)
    
            

# Add unmarked PROV
add_unmarked(marked, entity)
add_unmarked(marked, activity)
add_unmarked(marked, used)
add_unmarked(marked, was_derived_from)
add_unmarked(marked, was_generated_by)
add_unmarked(marked, had_member)
#print("\n".join(prov.RESULT))
pprint(prov.stats("../mutable_prov/unfold/result", "provn svg"))

In [ ]:
marked

In [ ]:
m = 10000 # max value
result = dist = [
    [0, 1, 4],
    [m, 0, 2],
    [2, m, 0],
]
nodes = len(dist)
indexes = set(range(nodes))
for k in indexes:
    distk = dist[k]
    for i in indexes - {k}:
        disti = dist[i]
        for j in indexes - {k, i}:
            print(k, i, j)
            ikj = disti[k] + distk[j]
            if disti[j] > ikj:
                disti[j] = ikj
print(result[0][2])

In [ ]:
range(nodes)

In [ ]:
indexes

In [ ]:
indexes = list(range(nodes))

In [ ]:
indexes[:k] + indexes[k + 1:] 

In [ ]:
indexes = set(range(nodes))

In [ ]:
indexes.discard(1)